In [2]:
import os
import sys
import gzip
import glob
import math
import numpy as np
import pandas as pd
from time import time
from scipy import stats
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.backends.backend_pdf
# matplotlib.style.use('ggplot')
%matplotlib inline

In [7]:
''' Loading data sets
'''
meta = pd.read_csv('data/metaData_taxistandsID_name_GPSlocation.csv')
train = pd.read_csv('data/train.1k.csv', quotechar='"')
print meta.head()
train.head()

   ID  Descricao       Latitude  Longitude
0   1       Agra  41.1771457135  -8.609670
1   2    Alameda    41.15618964  -8.591064
2   3     Aldoar  41.1705249231  -8.665876
3   4  Alfândega  41.1437639911  -8.621803
4   5      Amial  41.1835097223  -8.612726


,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."


In [27]:
print 'Distribution of CALL_TYPE'
print train.groupby('CALL_TYPE').apply(lambda x: len(x))
print 'Distribution of DAY_TYPE'
print train.groupby('DAY_TYPE').apply(lambda x: len(x))
print 'Distribution of MISSING_DATA'
print train.groupby('MISSING_DATA').apply(lambda x: len(x))

Distribution of CALL_TYPE
CALL_TYPE
A            267
B            433
C            299
dtype: int64
Distribution of DAY_TYPE
DAY_TYPE
A           999
dtype: int64
Distribution of MISSING_DATA
MISSING_DATA
False           999
dtype: int64


In [54]:
import ast
from ast import literal_eval
string = train[train.TRIP_ID==1372636858620000589].POLYLINE.values[0]
ll = list(literal_eval(string))
print ll
ll[0]
print ll[0][1] > ll[0][0]

[[-8.618643, 41.141412], [-8.618499, 41.141376], [-8.620326, 41.14251], [-8.622153, 41.143815], [-8.623953, 41.144373], [-8.62668, 41.144778], [-8.627373, 41.144697], [-8.630226, 41.14521], [-8.632746, 41.14692], [-8.631738, 41.148225], [-8.629938, 41.150385], [-8.62911, 41.151213], [-8.629128, 41.15124], [-8.628786, 41.152203], [-8.628687, 41.152374], [-8.628759, 41.152518], [-8.630838, 41.15268], [-8.632323, 41.153022], [-8.631144, 41.154489], [-8.630829, 41.154507], [-8.630829, 41.154516], [-8.630829, 41.154498], [-8.630838, 41.154489]]
True


In [60]:
print ll
print ll[-1]

[[-8.618643, 41.141412], [-8.618499, 41.141376], [-8.620326, 41.14251], [-8.622153, 41.143815], [-8.623953, 41.144373], [-8.62668, 41.144778], [-8.627373, 41.144697], [-8.630226, 41.14521], [-8.632746, 41.14692], [-8.631738, 41.148225], [-8.629938, 41.150385], [-8.62911, 41.151213], [-8.629128, 41.15124], [-8.628786, 41.152203], [-8.628687, 41.152374], [-8.628759, 41.152518], [-8.630838, 41.15268], [-8.632323, 41.153022], [-8.631144, 41.154489], [-8.630829, 41.154507], [-8.630829, 41.154516], [-8.630829, 41.154498], [-8.630838, 41.154489]]
[-8.630838, 41.154489]


In [66]:
# find BBox
lat = ll[-1][0]
lng = ll[-1][1]
# voting area
area = [0,0,0,0]
for coo in ll[:-1]:
    if coo[0] < lat and coo[1] < lng:
        area[0] += 1
    elif coo[0] < lat and coo[1] > lng:
        area[1] += 1
    elif coo[0] > lat and coo[1] < lng:
        area[2] += 1
    elif coo[0] > lat and coo[1] > lng:
        area[3] += 1
print area
areaArray = np.array(area)
idx = int(np.where(areaArray == areaArray.min())[0])
print idx, lat, lng

[3, 0, 14, 3]
1 -8.630838 41.154489
